In [24]:
import lightgbm
import pandas as pd

In [34]:
data = pd.read_csv("../../Data/LWB_Monthly_data.csv")

In [35]:
from tsfresh import extract_features
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters, EfficientFCParameters
from tsfresh.utilities.distribution import MultiprocessingDistributor
import re
import numpy as np

In [36]:
#feature = init_record.drop(["response_variable"], axis=1)

In [37]:
#label = init_record["response_variable"]
init_record = data
print(init_record.shape)

(123311, 341)


# Preprocess feature column name

In [38]:
init_record.columns = [re.sub(r"__*", "_", name) for name in init_record.columns]

# Encode id and time to numbers for tsfresh rollout

In [39]:
client_id = sorted(list(set(init_record["_key_client_id"])))
times = sorted(list(set(init_record["_key_occurreddate_month"])))
#print(times)
print(client_id)


['0000D172-EA88-432F-8235-9FAA00D29072', '0001AD9C-808F-4468-A32D-AB0700E2F6E2', '000377E7-E2BE-E011-B4AF-B6A03279A8B3', '00083605-BF0D-4109-BB00-AA8101120642', '000A27CC-6DEC-4BED-A21F-A10A00E3C375', '000A5916-F6C6-47EA-A20C-A975009F1020', '001D6DB5-1D88-E011-A5C6-B6A03279A8B3', '00202CA2-0103-414E-8058-A89E00EFACC9', '00207A41-F47D-4082-B845-A93800F1B075', '002232C9-52A7-E011-B4AF-B6A03279A8B3', '0026FF6B-AAB2-4778-B89E-A44A00C9B4F7', '00295479-A971-4819-B243-AA0000F6F200', '002D791F-C0B4-49E5-9114-52570510C043', '0031F295-DA64-4FA9-A943-5574ACB7EF6D', '00331EE5-75C3-4DCE-AAA1-A20B00C28F7B', '00334CDC-9186-45F0-B386-03A9673C4C6F', '0039DFD2-29AC-45CB-89E8-A882007E52DE', '003AF441-B307-4B84-9187-AB4500854139', '003D0226-5DD5-400F-A289-AB1000F861B8', '004C0B75-E6D8-4DA8-ADBC-AACD00B9AFD0', '004E27AD-511A-4426-8FC8-A7A8009B1A0A', '00519A9C-5916-4883-B26A-A9E500DC665A', '00540FA9-06C4-422E-983E-A8E80114DA52', '0054BC27-0103-47F8-815B-A02A00C59FE8', '00574313-DAC9-4B7E-A9AA-A3E101180ABA',

In [41]:
def create_time_dict(df):
    times = sorted(list(set(df["_key_occurreddate_month"])))
    time_dict ={}
    count=1
    for i in times:
        time_dict[i] = count
        count+=1
    return time_dict
time_dict = create_time_dict(init_record)
print(time_dict)

def create_client_id_dict(df):
    client_id = sorted(list(set(df["_key_client_id"])))
    client_dict = {}
    count = 1
    for i in client_id:
        client_dict[i] = count
        count += 1
    return client_dict
client_dict = create_client_id_dict(init_record)


{'2019-04-01': 1, '2019-05-01': 2, '2019-06-01': 3, '2019-07-01': 4, '2019-08-01': 5, '2019-09-01': 6, '2019-10-01': 7, '2019-11-01': 8, '2019-12-01': 9, '2020-01-01': 10, '2020-02-01': 11, '2020-03-01': 12}


In [42]:
init_record["time"] = init_record["_key_occurreddate_month"].map(time_dict)

In [43]:
init_record["id"] = init_record["_key_client_id"].map(client_dict)

In [44]:
init_record = init_record.drop(["_key_occurreddate_month", "_key_client_id"], axis = 1)

In [45]:
feature = init_record.drop(["response_variable"], axis=1)
label = init_record["response_variable"]

In [47]:
feature
print(feature.shape)
print(len(label))

(123311, 340)
123311


# Start rollout

In [48]:
from tsfresh.utilities.dataframe_functions import roll_time_series

In [50]:
df_rolled = roll_time_series(feature, column_id="id", column_sort="time")

Rolling: 100%|██████████| 6/6 [02:37<00:00, 26.29s/it]


In [51]:
df_rolled

,incident_catgry_care_concern_piv,incident_catgry_medication_piv,incident_catgry_medical_piv,incident_catgry_piv,incident_catgry_death_piv,incident_catgry_property_and_vehicles_piv,incident_catgry_physical_and_sexual_assault_piv,incident_catgry_client_missing_piv,incident_catgry_standard_of_care_soc_piv,incident_catgry_absconding_minor_significant_qld_jurisdi_piv,...,rp_type_detail_rp_end_restricted_access_piv,rp_type_detail_rp_start_restricted_access_piv,rp_type_detail_rp_end_mechanical_restraint_piv,rp_type_detail_rp_start_chemical_restraint_psychotropic_medication_piv,rp_type_detail_rp_start_environmental_restraint_piv,rp_type_detail_rp_end_exclusionary_time_out_not_including_seclusion_piv,restrictivepractice_isemergency_false_piv,restrictivepractice_isemergency_true_piv,time,id
83068,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"(1, 1)"
96473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"(1, 2)"
96474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,"(1, 2)"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,"(1, 3)"
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,"(1, 3)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,"(13405, 10)"
720233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,7,"(13405, 10)"
720234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,"(13405, 10)"
720235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9,"(13405, 10)"


# Extract features

In [ ]:
# change settings to EfficientFCParameters()
Distributor = MultiprocessingDistributor(n_workers=32,
                                         disable_progressbar=False,
                                         progressbar_title="Feature Extraction")
settings = EfficientFCParameters()

df_features = extract_features(df_rolled, column_id="id", column_sort="time",default_fc_parameters=settings, distributor=Distributor)

In [ ]:
final_feature = df_features.reset_index()

In [ ]:
final_feature

In [ ]:
rev_time_dict = {value:key for key, value in time_dict.items()}
rev_client_dict = {value:key for key, value in client_dict.items()}

In [ ]:
rev_time_dict
rev_client_dict

In [ ]:
final_feature["_key_occurreddate_month"] = final_feature["level_1"].map(rev_time_dict)
final_feature["_key_client_id"] = final_feature["level_0"].map(rev_client_dict)

In [ ]:
final_feature = final_feature.drop(["level_0", "level_1"], axis = 1)

In [ ]:
final_feature["response_variable"] = label
final_feature["response_variable"]

In [ ]:
final_feature.to_csv("ts_rollout_monthly_data.csv")